In [1]:
import pandas as pd

train = pd.read_csv('../data/train/train.csv')

In [2]:
# 메뉴 카테고리

from typing import DefaultDict
import re

dic = DefaultDict(list)
for venue_item in train['영업장명_메뉴명'].unique():
    venue, item = venue_item.split('_')
    dic[venue].append(item)

CATS = [
 ("한식 메인", r"정식|국밥|비빔밥|냉면|된장찌개|찌개|탕|전골|우거지|미역국|갈비탕|양지|해장국|파전|전\b|갱시기"),
 ("서양식 메인·라이트", r"파스타|스파게티|리조|리조또|피자|스테이크|양갈비|샐러드|까르보나라|알리오|돈까스|플래터|platter"),
 ("아시아 면류", r"라면|우동|짜장|짬뽕|사발면|컵라면"),
 ("구이·BBQ·육류", r"구이|BBQ|삼겹|불고기|차돌|돈육(?!.*라면)|한우(?!.*미역국)|\([0-9]+g\)|AUS|호주산|US|미국산|NZ|뉴질랜드산"),
 ("라이스·덮밥·볶음밥", r"볶음밥|덮밥|공깃밥|햇반|주먹밥|돌솥비빔밥"),
 ("분식·스낵", r"떡볶이|어묵|핫도그|소시지|치킨(?!.*와인)|프라이|감자|무침|골뱅이|닭발|순대"),
 ("디저트·베이커리", r"아이스크림|스크림|쿠키|빵|페스츄리"),
 ("커피·차", r"아메리카노|라떼|커피|coffee|얼그레이|Regular Coffee"),
 ("무알코올 음료", r"콜라|코카콜라|스프라이트|제로|에이드|아이스티|식혜|미숫가루|생수"),
 ("맥주", r"카스|Cass|하이네켄|스텔라|버드와이저|맥주|테라"),
 ("소주·증류주", r"소주|참이슬|처음처럼|안동소주"),
 ("와인", r"와인|Gls\.|Glass|Cabernet|카베르네|Merlot|메를로|Syrah|쉬라|시라|Chardonnay|샤르도네|Sauvignon|소비뇽|Riesling|리슬링|Sileni|Mission|미션 ?서드"),
 ("칵테일·하이볼", r"칵테일|하이볼|토닉"),
 ("전통주·막걸리", r"막걸리"),
 ("패키지·단체·대관·서비스·옵션", r"단체|패키지|무제한|대여|대관|Conference|Convention|Ballroom|OPUS|룸 이용|잔디그늘집|의자.*추가|Open Food|오픈푸드|사리|추가|샷 추가|수저|컵|접시|쌈장|허브솔트|브런치|쌈야채|세트"),
]

def categorize_item(name:str) -> str:
    for cat, pat in CATS:
        if re.search(pat, name, flags=re.IGNORECASE):
            return cat
    return "기타"

In [3]:
# 특성 추가

def derive_flags(name:str):
    flags = dict(
        # 1. 행사성/대량 주문 신호 → 주말·저녁·성수기 수요 피크 설명.
        is_group=bool(re.search(r"단체|패키지|무제한|[0-9]+인|모둠", name)),
        # 2. 공간사용 연계 품목 → 외부 캘린더 없이 행사성 수요 포착.
        is_room_service=bool(re.search(r"Conference|Convention|Ballroom|룸|대여|대관|잔디그늘집", name, re.I)),
        # 3. 메인 종속 옵션 → 메인 판매량 비율/별도 타깃으로 예측 안정화.
        is_addon=bool(re.search(r"사리|추가|샷 추가|빵 추가|면 추가|고기추가", name)),
        # 4. 소모품/소스 → 테이블 회전·메인 판매의 보조 신호로 노이즈 제거.
        is_disposable_or_condiment=bool(re.search(r"수저|컵|접시|쌈장|허브솔트|소금", name)),
        # 5. ‘제로’ 무설탕 음료 속성 → 건강/다이어트 수요 클러스터 구분.
        is_zero_sugar=bool(re.search(r"제로", name)),
        # 6. HOT/핫 표기의 뜨거운 음료 → 겨울·한파 등 계절/기온 민감 수요 반영.
        drink_temp_hot=bool(re.search(r"\bHOT\b|핫", name, re.I)),
        # 7. ICE/아이스 표기의 차가운 음료 → 여름·한낮 강화 등 계절·시간대 효과 반영.
        drink_temp_ice=bool(re.search(r"\bICE\b|아이스", name, re.I)),
        # 8. 맵기 속성 → 날씨·주류 동반구매와의 상호작용으로 시간대·요일 수요 설명.
        spicy_flag=bool(re.search(r"매콤|매운|마라|덜매운", name)),
        # 9. ‘어린이’ 대상 메뉴 → 주말·공휴일·방학의 패밀리 방문 증가 포착.
        kids_flag=bool(re.search(r"어린이", name)),
    )
    return flags

In [4]:
rows = []
for venue, items in dic.items():
    for item in items:
        cat = categorize_item(item)
        flags = derive_flags(item)
        row = {'venue': venue, 'item': item, 'category': cat}
        row.update(flags)
        rows.append(row)
df = pd.DataFrame(rows)
df

,venue,item,category,is_group,is_room_service,is_addon,is_disposable_or_condiment,is_zero_sugar,drink_temp_hot,drink_temp_ice,spicy_flag,kids_flag
0,느티나무 셀프BBQ,1인 수저세트,패키지·단체·대관·서비스·옵션,True,False,False,True,False,False,False,False,False
1,느티나무 셀프BBQ,BBQ55(단체),구이·BBQ·육류,True,False,False,False,False,False,False,False,False
2,느티나무 셀프BBQ,"대여료 30,000원",패키지·단체·대관·서비스·옵션,False,True,False,False,False,False,False,False,False
3,느티나무 셀프BBQ,"대여료 60,000원",패키지·단체·대관·서비스·옵션,False,True,False,False,False,False,False,False,False
4,느티나무 셀프BBQ,"대여료 90,000원",패키지·단체·대관·서비스·옵션,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
188,화담숲카페,메밀미숫가루,무알코올 음료,False,False,False,False,False,False,False,False,False
189,화담숲카페,아메리카노 HOT,커피·차,False,False,False,False,False,True,False,False,False
190,화담숲카페,아메리카노 ICE,커피·차,False,False,False,False,False,False,True,False,False
191,화담숲카페,카페라떼 ICE,커피·차,False,False,False,False,False,False,True,False,False


In [5]:
# 저장

df.to_csv("../data/train/venue_item.csv", index=False)